<a href="https://colab.research.google.com/github/Olfeng-xalaz/SCM/blob/main/Spediteur_erweitert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip als Paketmanager
! pip install -q pyscipopt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 31.1 MB/s eta 0:00:00


In [ ]:
from pyscipopt import Model, quicksum

# Optimierungsmodell für Spediteur

In [ ]:
# Erstellen einer Modellinstanz
scip = Model()

# Indexmengen

In [ ]:
I = ['A1','A2','A3','A4','A5','A6',] # Menge der Aufträge
J = ['LKW1','LKW2','LKW3'] # Menge der LKWs

# Entscheidungsvariablen

In [ ]:
x = {} # Nehme ich meinen Auftrag an oder nicht?
for i in I:
  for j in J:
    x[i,j] = scip.addVar(vtype='B', name=f'x_{i}_{j}') # Boolean für "ja" oder "nein" pro LKW
print('Entscheidungsvariablen =', scip.getVars())

Entscheidungsvariablen = [x_A1_LKW1, x_A1_LKW2, x_A1_LKW3, x_A2_LKW1, x_A2_LKW2, x_A2_LKW3, x_A3_LKW1, x_A3_LKW2, x_A3_LKW3, x_A4_LKW1, x_A4_LKW2, x_A4_LKW3, x_A5_LKW1, x_A5_LKW2, x_A5_LKW3, x_A6_LKW1, x_A6_LKW2, x_A6_LKW3]


# Parameter

In [ ]:
db = {} # Deckungsbeitrag der Aufträge
db ['A1'] = 10
db ['A2'] = 15
db ['A3'] = 25
db ['A4'] = 5
db ['A5'] = 11
db ['A6'] = 8

v = {} # Volumen der Aufträge
v ['A1'] = 3
v ['A2'] = 4.5
v ['A3'] = 9.25
v ['A4'] = 1.2
v ['A5'] = 4
v ['A6'] = 2.7

cap = {} # Kapazität der LKWs
cap ['LKW1'] = 10
cap ['LKW2'] = 8
cap ['LKW3'] = 5

# Zielfunktion

In [ ]:
scip.setObjective(quicksum(db[i] * x[i,j] for i in I for j in J), sense = 'maximize') # Maximierung des GesamtDB

# Restriktionen

In [ ]:
for j in J: # Für jeden LKW muss gelten...
  scip.addCons(quicksum(v[i] * x [i,j] for i in I) <= cap [j], name = f'Laderaumvolumen_LKW_{j}') # Restriktion auf Kapazität des LKWs

for i in I: # Für jeden Auftrag muss gelten...
  scip.addCons(quicksum(x[i,j] for j in J) <= 1, name = f"Einmaliges_Aufladen {i}") # Restriktion auf einmaliges Aufladen jedes Auftrags

print("Nebenbedingung: ", scip.getConss())

Nebenbedingung:  [Laderaumvolumen_LKW_LKW1, Laderaumvolumen_LKW_LKW2, Laderaumvolumen_LKW_LKW3, Einmaliges_Aufladen A1, Einmaliges_Aufladen A2, Einmaliges_Aufladen A3, Einmaliges_Aufladen A4, Einmaliges_Aufladen A5, Einmaliges_Aufladen A6, Laderaumvolumen_LKW_LKW1, Laderaumvolumen_LKW_LKW2, Laderaumvolumen_LKW_LKW3, Einmaliges_Aufladen A1, Einmaliges_Aufladen A2, Einmaliges_Aufladen A3, Einmaliges_Aufladen A4, Einmaliges_Aufladen A5, Einmaliges_Aufladen A6]


# Berechnung des Ergebnisses

In [ ]:
scip.optimize()
status = scip.getStatus()
gap = scip.getGap()
print(f"Status des Solvers: {status} , GAP={gap}% \n")

if status == 'optimal':
  print("Lösung:")
  print("Zielfunktionswert (Deckungsbeitrag) = ",round(scip.getObjVal())) # Erzielter maximaler Deckungsbeitrag bei optimaler Verteilung der Aufträge
  for j in J:
    print(f'\n{j}:')
    for i in I:
      print(f'X_{i}_{j} =', round(scip.getVal(x[i,j]))) # Welche Aufträge in welche LKWs?
else:
  print("Keine Lösung gefunden!")


Status des Solvers: optimal , GAP=0.0% 

Lösung:
Zielfunktionswert (Deckungsbeitrag) =  64

LKW1:
X_A1_LKW1 = 0
X_A2_LKW1 = 0
X_A3_LKW1 = 1
X_A4_LKW1 = 0
X_A5_LKW1 = 0
X_A6_LKW1 = 0

LKW2:
X_A1_LKW2 = 0
X_A2_LKW2 = 0
X_A3_LKW2 = 0
X_A4_LKW2 = 1
X_A5_LKW2 = 1
X_A6_LKW2 = 1

LKW3:
X_A1_LKW3 = 0
X_A2_LKW3 = 1
X_A3_LKW3 = 0
X_A4_LKW3 = 0
X_A5_LKW3 = 0
X_A6_LKW3 = 0
